In [ ]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
import io
from googleapiclient.errors import HttpError
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

key_file_name = os.getenv('KEY_FILE_NAME')
folder_id = os.getenv('FOLDER_ID')

In [3]:
# It controls the access of the application. Currently it allows read and write both
scope = ['https://www.googleapis.com/auth/drive']

current_directory = os.getcwd()
service_account_json_key = os.path.join(current_directory, key_file_name)

credentials = service_account.Credentials.from_service_account_file(
                              filename=service_account_json_key, 
                              scopes=scope)

service = build('drive', 'v3', credentials=credentials)

In [ ]:
page_token = None

# Google drive can serve upto 1000 files in one request, page solves this issue by accessing the next in the order
while True:
    results = service.files().list(
        pageSize=1000,  
        fields="nextPageToken, files(id, name)",
        q=f"'{folder_id}' in parents",
        pageToken=page_token
    ).execute()

    files = results.get('files', [])
    
    if not files:
        print(f'No more files found in folder with ID: {folder_id}')
        break

    # Print the name of files in the folder
    for file in files:
        print(file)

    # Get the next page token
    page_token = results.get('nextPageToken')

    if not page_token:
        break